In [47]:
import numpy as np
from numba import cuda

In [48]:
#INITIALIZER
x = 1024
#x = 20480
n = x * x #total elements in an integer array

original_array = np.arange(n).reshape((1024, 1024)).astype(np.int32)
cpu_res = np.zeros_like(original_array).astype(np.int32)
gpu_res = np.zeros_like(original_array).astype(np.int32)

arr = cuda.to_device(original_array)
gpu_res = cuda.to_device(gpu_res)

CudaAPIError: [700] Call to cuMemAlloc results in UNKNOWN_CUDA_ERROR

In [5]:
threads_per_block = (64, 64)
blocks = (int(x / 64), int(x / 64))

In [6]:
def cpu_func_(sol, arr, m, n):
    for i in range(m): #no. of rows
        for j in range(n): #no. of half columns
                       sol[i][j] = arr[i][n - j - 1]

In [25]:
@cuda.jit
def gpu_func_(sol, arr, n):
    share = cuda.shared.array((64, 65), numba_types.int32) #shared memory bank conflict handled
    x, y = cuda.grid(2)
    share[y][x] = arr[y][x]
    cuda.syncthreads()
    sol[y][n - x] = share[y][x]

In [44]:
cpu_func_(cpu_res, original_array, x, x)

In [26]:
gpu_func_[blocks, threads_per_block](gpu_res, arr, x)
cuda.synchronize()

sum_from_gpu = res[0]

CudaAPIError: [700] Call to cuLinkCreate results in UNKNOWN_CUDA_ERROR

# FOR CPU

In [45]:
%timeit cpu_func_(cpu_res, original_array, x, x)

549 ms ± 9.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# FOR GPU

In [21]:
%timeit gpu_func_[blocks, threads_per_block](gpu_res, arr, x); 

CudaAPIError: [700] Call to cuLinkCreate results in UNKNOWN_CUDA_ERROR

# COMPARING RESULTS

In [17]:
cpu_res == gpu_res

/home/appuser/Miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [46]:
print("CPU swapped array = \n", cpu_res)
print("GPU swapped array = \n", gpu_res.copy_to_host())

CPU swapped array = 
 [[   1023    1022    1021 ...       2       1       0]
 [   2047    2046    2045 ...    1026    1025    1024]
 [   3071    3070    3069 ...    2050    2049    2048]
 ...
 [1046527 1046526 1046525 ... 1045506 1045505 1045504]
 [1047551 1047550 1047549 ... 1046530 1046529 1046528]
 [1048575 1048574 1048573 ... 1047554 1047553 1047552]]


AttributeError: 'numpy.ndarray' object has no attribute 'copy_to_host'